In [1]:
import numpy as np

from astropy.io import fits
from astropy.table import Table

from astroML.density_estimation import XDGMM

## Load Data

In [2]:
# t = Table.read('kounkel_2019_apogee.csv')

In [3]:
# flag1 = t['e_RV'] < 0.5 #error cut on radial velocity
# flag2 = (t['Vrot'] > 5) & (t['Vrot'] < 150) #cut on vsini 
# flag3 = t['SB2']==1 #flag to ensure star is not a binary
# flag4 = t['YSO'] > 0 #cut to make sure the star is a YSO

# flag5_ic = (t['RV'] > 13) & (t['RV']<17) #cut on velocity outliers -- can be tweaked
# flag5_ngc = (t['RV'] > 12.5) & (t['RV'] < 17.5)

# flag6 = (t['parallax'] > 2.5) & (t['parallax'] < 4)
# #flag6 = potential cut on distance from cluster center?

In [4]:
# ic_mask = fits.getdata("ic348_kounkel_mask.fits").astype(bool) # cut to focus on IC348 field
# ngc_mask = fits.getdata("ngc1333_kounkel_mask.fits").astype(bool)

# flags = flag1 & flag2 & flag3 & flag4 
# # & flag6
# ic_flags = flags & flag5_ic & ic_mask
# ngc_flags = flags & flag5_ngc & ngc_mask

# data_ic = t[ic_flags]
# data_ngc = t[ngc_flags]

# data_ic = data_ic.filled()
# data_ngc = data_ngc.filled()

In [5]:
data_ic = Table.read("data/ic348_apogee.csv")
data_ngc = Table.read("data/ngc1333_apogee.csv")

## Run Deconvolution

#### IC 348
$\textit{set up array}$

In [6]:
X_ic = np.vstack([data_ic['RV']]).T
Xerr_ic = np.zeros(X_ic.shape + X_ic.shape[-1:])

for i in range(0,data_ic['e_RV'].shape[0]):
    np.fill_diagonal(Xerr_ic[i],[data_ic['e_RV'][i]**2])

$\textit{run deconvolution}$

In [7]:
clf_ic = XDGMM(n_components = 1)
clf_ic.fit(X_ic, Xerr_ic)

mu_ic = clf_ic.mu
cov_ic = np.diagonal(clf_ic.V)
sigma_ic = np.sqrt(np.diagonal(clf_ic.V))

#### NGC 1333
$\textit{set up array}$

In [8]:
X_ngc = np.vstack([data_ngc['RV']]).T
Xerr_ngc = np.zeros(X_ngc.shape + X_ngc.shape[-1:])

for i in range(0,data_ngc['e_RV'].shape[0]):
    np.fill_diagonal(Xerr_ngc[i],[data_ngc['e_RV'][i]**2])

$\textit{run deconvolution}$

In [9]:
clf_ngc = XDGMM(n_components = 1)
clf_ngc.fit(X_ngc, Xerr_ngc)

mu_ngc = clf_ngc.mu
cov_ngc = np.diagonal(clf_ngc.V)
sigma_ngc = np.sqrt(np.diagonal(clf_ngc.V))

## Summary 

In [10]:
print("IC 348 Dispersion")
print(f"Rvel mean    : {mu_ic[0]}")
print(f"Rvel sigma   : {sigma_ic[0]}")

print("NGC 1333 Dispersion")
print(f"Rvel mean    : {mu_ngc[0]}")
print(f"Rvel sigma   : {sigma_ngc[0]}")

IC 348 Dispersion
Rvel mean    : [15.16084675]
Rvel sigma   : [0.70639542]
NGC 1333 Dispersion
Rvel mean    : [14.21082717]
Rvel sigma   : [0.97002444]


In [11]:
# Cottaar et al (2015), IC 348 : 0.72 ± 0.07 km/s
# Foster et al (2015), NGC 1333 : 0.92 ± 0.12 km/s